# Adipocytes Tools

The **Adipocytes Tools** help to analyze fat cells in images from histological sections such as this one: [0178_x5_3.tif](http://dev.mri.cnrs.fr/attachments/190/0178_x5_3.tif). 

In [4]:
image = read_image_from_url('http://dev.mri.cnrs.fr/attachments/190/0178_x5_3.tif')
imshow(image, width=500, height=300)

The source code in git-hub can be found [here](https://github.com/MontpellierRessourcesImagerie/imagej_macros_and_scripts/tree/master/volker/toolsets/adipocyte_tools).

In [6]:
%%file Adipocyte_Tools.ijm
/**
  * MRI Adipocyte Tools
  * 
  * The Adipocytes Tools help to analyze fat cells in images from histological sections.
  * 
  * (c) 2020, INSERM
  * written by Volker Baecker at Montpellier Ressources Imagerie, Biocampus Montpellier, INSERM, CNRS, University of Montpellier (www.mri.cnrs.fr)
  * 
  * in collaboration with Matthieu Lacroix and Patricia Cavelier
  */

var helpURL = "https://github.com/MontpellierRessourcesImagerie/imagej_macros_and_scripts/wiki/Adipocytes-Tools"

// Preprocessing options
var preMinSize = 80;
var preMaxSize = 20000;
var preThresholdMethod = "Percentile";
var preNumberOfDilates = 10;
var preRemoveScale = true;

// Simple segmentation options
var simpleMinSize = 40;
var simpleMaxSize = 6000;
var simpleThresholdMethod = "Huang";
var simpleUseBinaryWatershed = true;
var simpleFindEdges = true;
var simpleClearBackground = true;
var simpleRemoveScale = true

// Watershed segmentation options
var waterMinSize = 50;
var waterMaxSize = 20000;
var waterSigma = 4;
var waterFindEdges = true;
var waterClearBackground = true;
var waterRemoveScale = true;

// Large magnification segmentation options
var largeMinSize = 10000;
var largeMaxSize = "Infinity";
var largeRemoveScale = true;
var largeNumberOfErodes = 3;

// Internal variables to set and reset the colors
var oldForeground;
var oldBackground;

// run a function here:

exit();
// Main functions start

// <p> - preprocessing clear background
function preProcessImage() {
    roiManager("reset");
    storeColors();
    setWhiteOnBlack();
    if (preRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
    clearBackground(preMinSize, preMaxSize, preThresholdMethod, preNumberOfDilates, false);
    resetColors();
    setBatchMode("exit and display");
    if (isOpen("Results")) {
		selectWindow("Results");
		run("Close");
    }
}

function clearBackground(minSize, maxSize, thresholdMethod,  numberOfDilates, keepSelection) {
	run("Duplicate...", " ");
    saveSettings();
    setOption("black background", false);
    title = getTitle();
    run("Find Edges");
    run("8-bit");
    run("Smooth");
    setAutoThreshold(thresholdMethod);
    run("Analyze Particles...", "size="+minSize+"-"+maxSize+" circularity=0.00-1.00 show=Masks exclude in_situ");
    run("Create Selection");
    run("Enlarge...", "enlarge=" + numberOfDilates + " pixel");
    roiManager("Add");
    close();
    roiManager("select", 0);
    run("Clear Outside");
    roiManager("reset");
    if (!keepSelection) run("Select None");
    restoreSettings();
}

// <s> - simple adipocytes segmentation
function simpleSegmentation() {
	run("Duplicate...", " ");
	run("ROI Manager...");
    roiManager("Show All with labels");
    setBatchMode(true);
    roiManager("reset");
    storeColors();
    setWhiteOnBlack();
    if (simpleRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
    if (simpleUseBinaryWatershed && simpleClearBackground) clearBackground(preMinSize, preMaxSize, preThresholdMethod, preNumberOfDilates, true);
    if (simpleRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
    if (simpleFindEdges) run("Find Edges");
    run("8-bit");
    if (simpleFindEdges) {
        run("Smooth");
        run("Invert");
    }
    setAutoThreshold(simpleThresholdMethod+ " dark");
    run("Convert to Mask");
    if (simpleUseBinaryWatershed) run("Watershed");
    run("Clear Results");
    run("Analyze Particles...", "size="+simpleMinSize+"-"+simpleMaxSize+" circularity=0.00-1.00 show=Nothing add exclude");
    close();
    if (simpleRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
    resetColors();
    setBatchMode("exit and display");
    if (isOpen("Results")) {
	    selectWindow("Results");
	    run("Close");
   }
   roiManager("Show All with labels");
}

// <w> - watershed adipocytes segmentation
function watershedAdipocytesSegmentation() {
   run("Duplicate...", " ");	
   run("ROI Manager...");
   roiManager("Show All with labels");
   setBatchMode(true);
   roiManager("reset");
   storeColors();
   setWhiteOnBlack();
   if (waterRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
   if (waterClearBackground) clearBackground(preMinSize, preMaxSize, preThresholdMethod, preNumberOfDilates, false);
   title = getTitle();
   if (waterRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
   if (waterFindEdges) run("Find Edges");
   else  run("Invert");
   run("8-bit");
   run("Gaussian Blur...", "sigma=" + waterSigma);
   run("Watershed Algorithm");
   run("Invert");
   imageCalculator("AND create", title,"Watershed");
   titleResult = getTitle();
   selectImage("Watershed");
   close();
   selectImage(titleResult);
   setThreshold(1, 255);
   run("Convert to Mask");
   run("Clear Results");
   run("Analyze Particles...", "size="+waterMinSize+"-"+waterMaxSize+" circularity=0.00-1.00 show=Nothing add exclude");
   selectImage(titleResult); 
   close();
   close();
   if (waterRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
   resetColors();
   setBatchMode("exit and display");
   if (isOpen("Results")) {
        selectWindow("Results");
        run("Close");
   }
   roiManager("Show All with labels");
}

// <l> - large magnification adipocytes segmentation
function largeMagnificationSegmentation() {
   run("Duplicate...", " ");	
   run("ROI Manager...");
   roiManager("Show All with labels");
   setBatchMode(true);
   roiManager("reset");
   storeColors();
   setWhiteOnBlack();
   if (largeRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
   title = getTitle();
   
    run("Fit Polynomial", "x=2 y=2 mixed=1");
    run("8-bit");
    setAutoThreshold("Huang dark");
    setOption("BlackBackground", false);
    run("Convert to Mask");
    for (i=0; i<largeNumberOfErodes; i++) {
	    run("Erode");
    }
    run("Fill Holes");
    run("Create Selection");
    run("Crop");
    run("Select None");
    run("Analyze Particles...", "size="+largeMinSize+"-"+largeMaxSize+" exclude add");

   close();
   if (largeRemoveScale) run("Set Scale...", "distance=0 known=0 pixel=1 unit=pixel");
   resetColors();
   setBatchMode("exit and display");
   if (isOpen("Results")) {
        selectWindow("Results");
        run("Close");
   }
   roiManager("Show All with labels");
}
// Main functions end

// Macros tools
macro "Adipocytes Tools Help (f1) Action Tool - C98aD00D01D02D03D04D05D06D07D08D09D0aD4fDd0De0Df8Df9CfffD1dD29D2eD34D49D51D56D6cD85D8bD9aDa1Dc6DdaDe2CddeD14D15D16D17D18D1cD2aD2dD4aD5dD5eD62D65D66D67D68D72D75D76D77D82D83D84D8aD92D93D94D98D99Da2Da3Da7Da8Da9Db1Db7Db8Db9Dc7Dc8Dc9CfffD0bD0cD0dD0eD0fD10D1eD1fD20D24D25D26D27D28D2fD30D35D36D37D38D39D3fD40D45D46D47D48D50D57D58D5fD60D61D6dD6eD6fD70D71D7cD7dD7eD7fD80D81D8cD8dD8eD8fD90D91D95D96D9bD9cD9dD9eD9fDa0Da5Da6DaaDabDacDadDaeDafDb0Db5Db6DbaDbbDbcDbdDbeDbfDc0DcaDcbDccDcdDceDcfDdbDdcDddDdeDdfDebDecDedDeeDefDf0Df1Df2Df3Df4Df5Df6Df7DfaDfbDfcDfdDfeDffCccdD13D19D21D22D31D33D3eD42D43D4eD52D5aD5cD63D6bD73D78D7aD87Db3Dc1Dc4Dd1Dd2Dd3Dd5Dd6Dd9De4De5De6CeeeD23D3aD41D44D55D59D7bD86D97Da4Db4Dc5De1De3DeaCcbcD1aD1bD2bD2cD32D3bD4bD4dD53D54D5bD64D69D6aD74D79D88D89Db2Dc2Dc3Dd4Dd7Dd8De7De9CaabD11D12D3cD3dD4cDe8"{
	run('URL...', 'url='+helpURL);
}

macro "Preprocessing Clear Background (f2) Action Tool - C000T4b12p" {
	preProcessImage();
}

macro "Simple Adipocytes Segmentation (f3) Action Tool- C000T4b12s" {
	simpleSegmentation();
}

macro "Watershed Adipocytes Segmentation (f4) Action Tool- C000T4b12w" {
	watershedAdipocytesSegmentation();
}

macro "Large Magnification Adipocytes Segmentation (f5) Action Tool- C000T4b12l" {
	largeMagnificationSegmentation();
}

// Extras menu
   var extraCmds = newMenu("Extras Menu Tool",
      newArray("Download example image","Open help page"));
      
   macro "Extras Menu Tool - C000T4b12e" {
       cmd = getArgument();
       if (cmd=="Download example image")
           open("http://dev.mri.cnrs.fr/attachments/190/0178_x5_3.tif");
       else if (cmd=="Open help page")
           run('URL...', 'url='+helpURL);
   }

// keyboard shortcuts
macro 'Adipocytes Tools Help [f1]' {
	 run('URL...', 'url='+helpURL);
}

macro 'Preprocessing Clear Background [f2]' {
	preProcessImage();
}

macro 'Simple Adipocytes Segmentation [f3]' {
	simpleSegmentation();
}

macro 'Watershed Adipocytes Segmentation [f4]' {
	watershedAdipocytesSegmentation();
}

macro 'Large Magnification Adipocytes Segmentation [f5]' {
	largeMagnificationSegmentation();
}

// Option dialogs
macro "Preprocessing Clear Background (f2) Action Tool Options" {
        Dialog.create("Preprocessing Adipocytes Segmentation Options");
        Dialog.addNumber("min. size", preMinSize);	
        Dialog.addNumber("max. size", preMaxSize);
        Dialog.addNumber("nr. of dilates", preNumberOfDilates);
        Dialog.addChoice("thresholding method", newArray("Default", "Huang", "Intermodes", "IsoData", "IJ_IsoData", "Li", "MaxEntropy", "Mean", "MinError",  "Minimum", "Moments",  "Otsu", "Percentile", "RenyiEntropy", "Shanbhag", "Triangle", "Yen"),  preThresholdMethod);
        Dialog.addCheckbox("remove scale", preRemoveScale);
        Dialog.show();
        preMinSize = Dialog.getNumber();
        preMaxSize = Dialog.getNumber();
        preNumberOfDilates = Dialog.getNumber();
        preThresholdMethod = Dialog.getChoice();
        preRemoveScale = Dialog.getCheckbox();
}

macro "Simple Adipocytes Segmentation (f3) Action Tool Options" {
       Dialog.create("Simple Adipocytes Segmentation Options");
       Dialog.addNumber("min. size", simpleMinSize);	
       Dialog.addNumber("max. size", simpleMaxSize);
       Dialog.addChoice("thresholding method", newArray("Default", "Huang", "Intermodes", "IsoData", "IJ_IsoData", "Li", "MaxEntropy", "Mean", "MinError",  "Minimum", "Moments",  "Otsu", "Percentile", "RenyiEntropy", "Shanbhag", "Triangle", "Yen"),  simpleThresholdMethod);
       Dialog.addCheckbox("clear background", simpleClearBackground);
       Dialog.addCheckbox("find edges", simpleFindEdges);
       Dialog.addCheckbox("use binary watershed", simpleUseBinaryWatershed);
       Dialog.addCheckbox("remove scale", simpleRemoveScale);
       Dialog.show();
       simpleMinSize = Dialog.getNumber();
       simpleMaxSize = Dialog.getNumber();
       simpleThresholdMethod = Dialog.getChoice();
       simpleClearBackground = Dialog.getCheckbox();
       simpleFindEdges = Dialog.getCheckbox();
       simpleUseBinaryWatershed = Dialog.getCheckbox();
       simpleRemoveScale = Dialog.getCheckbox();
}

macro "Watershed Adipocytes Segmentation (f4) Action Tool Options" {
        Dialog.create("Watershed Adipocytes Segmentation Options");
        Dialog.addNumber("min. size", waterMinSize);	
        Dialog.addNumber("max. size", waterMaxSize);
        Dialog.addNumber("sigma", waterSigma);
        Dialog.addCheckbox("clear background", waterClearBackground);
        Dialog.addCheckbox("find edges", waterFindEdges);
        Dialog.addCheckbox("remove scale", waterRemoveScale);
        Dialog.show();
        waterMinSize = Dialog.getNumber();
        waterMaxSize = Dialog.getNumber();
        waterSigma = Dialog.getNumber();
        waterClearBackground = Dialog.getCheckbox();
        waterFindEdges = Dialog.getCheckbox();
        waterRemoveScale = Dialog.getCheckbox();
}

macro "Large Magnification Adipocytes Segmentation (f5) Action Tool Options" {
        Dialog.create("Large Magnification Adipocytes Segmentation Options");
        Dialog.addNumber("min. size", largeMinSize);	
        Dialog.addNumber("max. size", largeMaxSize);
        Dialog.addNumber("nr. of erodes", largeNumberOfErodes);
        Dialog.addCheckbox("remove scale", largeRemoveScale);
        Dialog.show();
        largeMinSize = Dialog.getNumber();
        largeMaxSize = Dialog.getNumber();
        largeNumberOfErodes = Dialog.getNumber();
        largeRemoveScale = Dialog.getCheckbox();
}

// helper functions
function storeColors() {
       oldForeground = getValue("color.foreground");
       oldBackground = getValue("color.background");
}

function setWhiteOnBlack() {
    setForegroundColor(255,255,255);
    setBackgroundColor(0,0,0);
}

function setBlackOnWhite() {
    setForegroundColor(0,0,0);
    setBackgroundColor(255,255,255);
}

function resetColors() {
    setForegroundColor((oldForeground>>16)&0xff, (oldForeground>>8)&0xff, oldForeground&0xff);
    setBackgroundColor((oldBackground>>16)&0xff, (oldBackground>>8)&0xff, oldBackground&0xff);
}


Overwriting Adipocyte_Tools.ijm


# Installation in FIJI/ImageJ

To install the tools, drag the link [Adipocyte_Tools.ijm](https://raw.githubusercontent.com/MontpellierRessourcesImagerie/imagej_macros_and_scripts/master/volker/toolsets/adipocyte_tools/Adipocyte_Tools.ijm) to the ImageJ launcher window and save it under _macros/toolsets_ in the ImageJ installation. In order to use the greyscale-watershed method (the **w**-button) you must have the [Watershed Algorithm](http://rsbweb.nih.gov/ij/plugins/watershed.html) installed. 

Select the "MRI Adipocytes Tools" toolset from the ``>>`` button of the ImageJ launcher.

![toolset](http://dev.mri.cnrs.fr/attachments/download/1691/toolset.png)

* the first button (the one with the image) opens the help page
* the p button runs a preprocessing step that clears the background of the image
* the s button runs a simple segmentation algorithm on the current image
* the w button runs a greyscale-watershed segmentation on the current image
* the l button runs a segmentation for large-magnification images on the current image

# Preprocessing

In the preprocessing step the background, which is free of cells is removed. The preprocessing is used in the first two segmentation methods, when "watershed" is used. Without it, the watershed-algorithm would split the background into objects as well.

```python
    run("Duplicate...", " ");
    run("Find Edges");
    run("8-bit");
    run("Smooth");
    setAutoThreshold(thresholdMethod);
    run("Analyze Particles...", "size="+minSize+"-"+maxSize+" circularity=0.00-1.00 show=Masks exclude in_situ");
    run("Create Selection");
    run("Enlarge...", "enlarge=" + numberOfDilates + " pixel");
    roiManager("Add");
    close();
    roiManager("select", 0);
    run("Clear Outside");
```



In [7]:
%%groovy
import com.twosigma.beakerx.widget.*

void runPreprocessing(minSize, maxSize, nrOfDilates, thresholdingMethod, removeScale) {
    macro = new File('Adipocyte_Tools.ijm').text
    macro = macro.replaceFirst('var preMinSize = 80;', 'var preMinSize = '+minSize+';')
    macro = macro.replaceFirst('var preMaxSize = 20000;', 'var preMaxSize = '+maxSize+';')
    macro = macro.replaceFirst('var preNumberOfDilates = 10;', 'var preNumberOfDilates = '+nrOfDilates+';')
    macro = macro.replaceFirst('var preThresholdMethod = "Percentile";', 'var preThresholdMethod = "'+thresholdingMethod+'";')
    macro = macro.replaceFirst('var preRemoveScale = true;', 'var preRemoveScale = '+removeScale+';')
    
    macro = macro.replaceFirst('// run a function here:', 'preProcessImage();\nsave("adipos_background_removed.tif");')
    
    ij.IJ.open('http://dev.mri.cnrs.fr/attachments/190/0178_x5_3.tif')
    ij.IJ.runMacro(macro)
    ij.IJ.run("Close All", "")
}
    
PF = new EasyForm("Preprocessing")

PF_MIN_SIZE = new IntText()
PF_MIN_SIZE.value = 80
PF_MIN_SIZE.description="min. size:"

PF_MAX_SIZE = new IntText()
PF_MAX_SIZE.value = 20000
PF_MAX_SIZE.description="max. size:"

PF_NR_OF_DILATES = new IntText()
PF_NR_OF_DILATES.value = 10
PF_NR_OF_DILATES.description = 'nr. of dilates:'

THRESHOLDING_METHODS = ['Default', 'Huang', 'Intermodes', 'IsoData', 'IJ_IsoData', 'Li', 'MaxEntropy', 'Mean', 'MinError', 'Minimum', 'Moments', 'Otsu', 'Percentile', 'RenyiEntropy', 'Shanbhag', 'Triangle', 'Yen']
PF.addWidget("min. size", PF_MIN_SIZE)
PF.addWidget("max. size", PF_MAX_SIZE)
PF.addWidget("nr. of dilates", PF_NR_OF_DILATES)
PF.addComboBox("thresholding method:", THRESHOLDING_METHODS)
PF.addCheckBox("remove scale")
PF_run_button = PF.addButton("run")
PF_run_button.actionPerformed = {
    runPreprocessing(PF_MIN_SIZE.value, 
                     PF_MAX_SIZE.value, 
                     PF_NR_OF_DILATES.value, 
                     PF['thresholding method:'],
                     PF['remove scale']
    )
}
PF['thresholding method:'] = 'Percentile'
PF['remove scale'] = 'true'
display(PF)

null

In [15]:
import tifffile as tiff
image = tiff.imread('adipos_background_removed.tif')
imshow(image, width=500, height=300)

In [1]:
%%groovy
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
%%classpath add mvn
net.imagej imagej-legacy 0.35.0
net.imagej imagej 2.0.0-rc-71

Groovy started successfully

Added new repo: scijava.public


In [2]:
from beakerx import *

def read_image_from_url(url):
    import requests
    import tifffile as tiff
    import io
    resp = requests.get(url)
    return tiff.imread(io.BytesIO(resp.content))

def imshow(image, width=400, height=300):
    import IPython
    import ipywidgets as widgets
    
    displayImage = get_display_image(image, width=width, height=height)

    SID_image = widgets.Image(
        value=displayImage.data.tobytes(),
        format='jpg',
        width=width,
        height=height,
    )
    SID_slider = widgets.IntSlider(
        min=100,
        max=1900,
        continuous_update=False,
        value = width,
        description = 'width',
        layout=Layout(width=str(width)+'px')
    )

    SID_link = widgets.jslink((SID_image, 'width'), (SID_slider, 'value'))    

    SID_form = EasyForm("Image display")
    SID_form.addWidget('FloatSlider', SID_slider)
    SID_form.addWidget('ImageDisplay', SID_image)
    display(SID_form)
    
def get_display_image(img, width=800, height=800):
    import cv2
    import IPython
    _,ret = cv2.imencode('.jpg', img) 
    i = IPython.display.Image(data=ret, width=width, height=height)
    return i


In [3]:
%%groovy
ij = new net.imagej.ImageJ()
"ImageJ v${ij.getVersion()} is ready to go."

ImageJ v2.0.0-rc-71/1.52i is ready to go.